In [0]:
from pyspark.sql import SparkSession
import re

# Inicializar a sessão Spark
spark = SparkSession.builder.appName("ReplicateFolderStructure").getOrCreate()

# Definir os caminhos de origem e destino
source_path = "/Volumes/ingestao_dados/tabnews/datalake_tabnews/"
destination_path = "/Volumes/ingestao_dados/trampar_lakehouse/novodatalake/TabNews"

def list_directories(path):
    """Lista todos os diretórios em um caminho, incluindo subdiretórios."""
    dirs = []
    for item in dbutils.fs.ls(path):
        if item.isDir():
            dirs.append(item.path)
            dirs.extend(list_directories(item.path))
    return dirs

def create_directory_structure(source_dir, dest_base):
    """Cria a estrutura de diretórios no destino."""
    # Remove o prefixo 'dbfs:' se estiver presente
    source_dir = source_dir.replace("dbfs:", "")
    dest_base = dest_base.replace("dbfs:", "")
    
    # Extrai o caminho relativo
    relative_path = source_dir.replace(source_path, "").lstrip("/")
    
    # Constrói o novo caminho de destino
    new_dir = f"{dest_base}/{relative_path}"
    
    try:
        dbutils.fs.mkdirs(new_dir)
        print(f"Diretório criado: {new_dir}")
    except Exception as e:
        print(f"Erro ao criar diretório {new_dir}: {str(e)}")

# Listar todos os diretórios na origem
source_dirs = list_directories(source_path)

# Criar a estrutura de diretórios no destino
for dir_path in source_dirs:
    create_directory_structure(dir_path, destination_path)

print("Processo de replicação da estrutura de pastas concluído.")

# Verificar a estrutura criada
print("\nEstrutura de pastas no destino:")
for item in dbutils.fs.ls(destination_path):
    print(item.path)

# Mostrar volumes disponíveis
print("\nVolumes disponíveis:")
spark.sql("SHOW VOLUMES IN ingestao_dados.trampar_lakehouse").show()

In [0]:
from pyspark.sql import SparkSession
import os

# Inicializar a sessão Spark
spark = SparkSession.builder.appName("CopyJSONFiles").getOrCreate()

# Definir os caminhos de origem e destino
source_path = "/Volumes/ingestao_dados/tabnews/datalake_tabnews"
destination_path = "/Volumes/ingestao_dados/trampar_lakehouse/novodatalake"

def list_json_files(path):
    """Lista todos os arquivos JSON em um caminho, incluindo subdiretórios."""
    json_files = []
    for item in dbutils.fs.ls(path):
        if item.isDir():
            json_files.extend(list_json_files(item.path))
        elif item.name.lower().endswith('.json'):
            json_files.append(item.path)
    return json_files

def copy_file(source, destination):
    """Copia um arquivo da origem para o destino."""
    try:
        dbutils.fs.cp(source, destination, True)  # True para sobrescrever
        print(f"Arquivo copiado: {source} -> {destination}")
        return True
    except Exception as e:
        print(f"Erro ao copiar {source} para {destination}: {str(e)}")
        return False

# Listar todos os arquivos JSON na origem
source_files = list_json_files(source_path)

# Copiar os arquivos
successful_copies = 0
for source_file in source_files:
    # Construir o caminho de destino
    relative_path = source_file.replace(source_path, "").lstrip("/")
    destination_file = os.path.join(destination_path, relative_path)
    
    if copy_file(source_file, destination_file):
        successful_copies += 1

print(f"Processo de cópia concluído. {successful_copies} arquivos copiados com sucesso.")

# Verificar o número de arquivos JSON no destino
def count_json_files(path):
    """Conta o número de arquivos JSON em um caminho, incluindo subdiretórios."""
    count = 0
    for item in dbutils.fs.ls(path):
        if item.isDir():
            count += count_json_files(item.path)
        elif item.name.lower().endswith('.json'):
            count += 1
    return count

num_files_destination = count_json_files(destination_path)
print(f"Número de arquivos JSON no destino: {num_files_destination}")